## 深入解析Pod对象（一）：基本概念

```txt
Pod 扮演的是传统部署环境里“虚拟机”的角色。这样的设计，是为了使用户从传统环境（虚拟机环境）向 Kubernetes （容器环境）的迁移，更加平滑。 而如果你能把 Pod 看成传统环境里的“机器”、把容器看作是运行在这个“机器”里的“用户程序”，那么很多关于 Pod 对象的设计就非常容易理解了。
```

* 凡是调度、网络、存储，以及安全相关的属性，基本上是 Pod 级别的。

```txt
这些属性的共同特征是，它们描述的是“机器”这个整体，而不是里面运行的“程序”。比如，配置这个“机器”的网卡（即：Pod 的网络定义），配置这个“机器”的磁盘（即：Pod 的存储定义），配置这个“机器”的防火墙（即：Pod 的安全定义）。这台“机器”运行在哪个服务器之上（即：Pod 的调度）。
```

### Pod 中几个重要字段的含义和用法

* NodeSelector：是一个供用户将 Pod 与 Node 进行绑定的字段

```yaml
apiVersion: v1
kind: Pod
...
spec:
 nodeSelector:
   disktype: ssd
```

> 这个 Pod 永远只能运行在携带了“disktype: ssd”标签（Label）的节点上；否则，它将调度失败。

* NodeName：一旦 Pod 的这个字段被赋值，Kubernetes 项目就会被认为这个 Pod 已经经过了调度，调度的结果就是赋值的节点名字。所以，这个字段一般由调度器负责设置，但用户也可以设置它来“骗过”调度器，当然这个做法一般是在测试或者调试的时候才会用到。

* HostAliases：定义了 Pod 的 hosts 文件（比如 /etc/hosts）里的内容。

```yaml
apiVersion: v1
kind: Pod
...
spec:
  hostAliases:
  - ip: "10.1.2.3"
    hostnames:
    - "foo.remote"
    - "bar.remote"
...
```

### 凡是跟容器的 Linux Namespace 相关的属性，也一定是 Pod 级别的。

> Pod 的设计，就是要让它里面的容器尽可能多地共享 Linux Namespace，仅保留必要的隔离和限制能力。这样，Pod 模拟出的效果，就跟虚拟机里程序间的关系非常类似了。

### Pod 中共享 PID Namespace

```bash
$ nano nginx.yaml
```

```yaml
apiVersion: v1
kind: Pod
metadata:
  name: nginx
spec:
  shareProcessNamespace: true
  containers:
  - name: nginx
    image: nginx
  - name: shell
    image: busybox
    stdin: true
    tty: true
```

> 在 Pod 的 YAML 文件里声明开启它们俩，其实等同于设置了 docker run 里的 -it（-i 即 stdin，-t 即 tty）参数。

```bash
$ kubectl apply -f nginx.yaml
$ kubectl attach -it nginx -c shell
/ # ps ax
PID   USER     TIME  COMMAND
    1 root      0:00 /pause
    8 root      0:00 nginx: master process nginx -g daemon off;
   16 101       0:00 nginx: worker process
   17 root      0:00 sh
   26 root      0:00 ps ax
```

### 凡是 Pod 中的容器要共享宿主机的 Namespace，也一定是 Pod 级别的定义

```yaml
apiVersion: v1
kind: Pod
metadata:
  name: nginx
spec:
  hostNetwork: true
  hostIPC: true
  hostPID: true
  containers:
  - name: nginx
    image: nginx
  - name: shell
    image: busybox
    stdin: true
    tty: true
```

> 共享宿主机的 Network、IPC 和 PID Namespace


### Containers

* Init Containers 的生命周期，会先于所有的 Containers，并且严格按照定义的顺序执行。

* ImagePullPolicy 字段。它定义了镜像拉取的策略。Always, Never, IfNotPresent。默认值：Always。

* Lifecycle 字段。它定义的是 Container Lifecycle Hooks。作用是在容器状态发生变化时触发一系列“钩子”。

```yaml
apiVersion: v1
kind: Pod
metadata:
  name: lifecycle-demo
spec:
  containers:
  - name: lifecycle-demo-container
    image: nginx
    lifecycle:
      postStart:
        exec:
          command: ["/bin/sh", "-c", "echo Hello from the postStart handler > /usr/share/message"]
      preStop:
        exec:
          command: ["/usr/sbin/nginx","-s","quit"]
```

### Pod 对象在 Kubernetes 中的生命周期。

#### Status

* Pending
* Running
* Succeeded
* Failed
* Unknown

#### Conditions Status的细分状态
* Initialized
* Ready
* ContainersReady
* PodScheduled